<a href="https://colab.research.google.com/github/Boonyaratt/LGBM_recommendation/blob/master/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from datetime import datetime, timedelta
from dateutil import easter
from typing import Dict, Text
import os, kagglehub
import lightgbm as lgb
import heapq
from functools import lru_cache
from collections import defaultdict
from xgboost import XGBRanker
import pathlib
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

np.random.seed(42)

In [6]:
!mkdir Dataset

mkdir: cannot create directory ‘Dataset’: File exists


In [7]:
os.environ["KAGGLEHUB_CACHE"] = "/content/Dataset"  # exact case
pathlib.Path("/content/Dataset").mkdir(parents=True, exist_ok=True)

kaggle_path = kagglehub.dataset_download("frtgnn/dunnhumby-the-complete-journey")
print(kaggle_path)

Using Colab cache for faster access to the 'dunnhumby-the-complete-journey' dataset.
/kaggle/input/dunnhumby-the-complete-journey


In [8]:
# path = "/kaggle/input/dunnhumby-the-complete-journey/"
path = "/content/Dataset/datasets/frtgnn/dunnhumby-the-complete-journey/versions/1/"

campaign_desc = pd.read_csv(path + "campaign_desc.csv")
campaign_table = pd.read_csv(path + "campaign_table.csv")
coupon_redempt = pd.read_csv(path + "coupon_redempt.csv") ## 1 counpon_unc can have multiple product_id: 556 nunique from 2318
coupon = pd.read_csv(path + "coupon.csv")
product = pd.read_csv(path + "product.csv")
transaction_data = pd.read_csv(path + "transaction_data.csv")
print("DataFrames loaded successfully!")
print(f"campaign_desc shape: {campaign_desc.shape}")
print(f"campaign_table shape: {campaign_table.shape}")
print(f"coupon_redempt shape: {coupon_redempt.shape}")
print(f"coupon shape: {coupon.shape}")
print(f"product shape: {product.shape}")
print(f"transaction_data shape: {transaction_data.shape}")

print("Headers for each dataset:")
print("=" * 50)

datasets = {
    'campaign_desc': campaign_desc,
    'campaign_table': campaign_table,
    'coupon_redempt': coupon_redempt,
    'coupon': coupon,
    'product': product,
    'transaction_data': transaction_data
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"Columns: {list(df.columns)}")

DataFrames loaded successfully!
campaign_desc shape: (30, 4)
campaign_table shape: (7208, 3)
coupon_redempt shape: (2318, 4)
coupon shape: (124548, 3)
product shape: (92353, 7)
transaction_data shape: (2595732, 12)
Headers for each dataset:

campaign_desc:
Columns: ['DESCRIPTION', 'CAMPAIGN', 'START_DAY', 'END_DAY']

campaign_table:
Columns: ['DESCRIPTION', 'household_key', 'CAMPAIGN']

coupon_redempt:
Columns: ['household_key', 'DAY', 'COUPON_UPC', 'CAMPAIGN']

coupon:
Columns: ['COUPON_UPC', 'PRODUCT_ID', 'CAMPAIGN']

product:
Columns: ['PRODUCT_ID', 'MANUFACTURER', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC', 'CURR_SIZE_OF_PRODUCT']

transaction_data:
Columns: ['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC']


##2

### ลดจำนวนข้อมูล
จำกัดวัน - จำนวนผู้ใช้ ที่ใช้ใน dataset

In [9]:
N_HH = 500
top_hh = (transaction_data["household_key"].value_counts().head(N_HH).index.astype(str))
transaction_data = transaction_data[transaction_data["household_key"].astype(str).isin(top_hh)].copy()
campaign_table   = campaign_table[campaign_table["household_key"].astype(str).isin(set(top_hh))].copy()
coupon_redempt   = coupon_redempt[coupon_redempt["household_key"].astype(str).isin(set(top_hh))].copy()

# ============================ #
WINDOW_DAYS = 180

min_day_all = int(transaction_data["DAY"].min()) #1
max_day_all = int(transaction_data["DAY"].max()) #711
start_day = max(min_day_all, max_day_all - WINDOW_DAYS + 1)
end_day   = max_day_all

keep_hh = set(transaction_data["household_key"].astype(str).unique())
campaign_table = campaign_table[campaign_table["household_key"].astype(str).isin(keep_hh)].copy()
coupon_redempt = coupon_redempt[coupon_redempt["household_key"].astype(str).isin(keep_hh)].copy()

# remove product_id not it transaction
keep_products = set(transaction_data["PRODUCT_ID"].astype(str).unique())
product = product[product["PRODUCT_ID"].astype(str).isin(keep_products)].copy()

In [10]:
print(f"Using DAY window [{start_day}, {end_day}]")

Using DAY window [532, 711]


In [11]:
print("transaction_data:", transaction_data.shape, "campaign_desc:", campaign_desc.shape,
      "campaign_table:", campaign_table.shape, "coupon_redempt:", coupon_redempt.shape,
      "coupon:", coupon.shape, "product:", product.shape)

transaction_data: (1326387, 12) campaign_desc: (30, 4) campaign_table: (3657, 3) coupon_redempt: (1656, 4) coupon: (124548, 3) product: (63047, 7)


In [12]:
max_day_all

711

###Change Data type


In [13]:
coupon['COUPON_UPC'] = coupon['COUPON_UPC'].astype(str)
coupon['CAMPAIGN'] = coupon['CAMPAIGN'].astype(str)
coupon['PRODUCT_ID'] = coupon['PRODUCT_ID'].astype(str)

campaign_desc['CAMPAIGN'] = campaign_desc['CAMPAIGN'].astype(str)

campaign_table['CAMPAIGN'] = campaign_table['CAMPAIGN'].astype(str)
campaign_table['household_key'] = campaign_table['household_key'].astype(str)

coupon_redempt['COUPON_UPC'] = coupon_redempt['COUPON_UPC'].astype(str)
coupon_redempt['CAMPAIGN'] = coupon_redempt['CAMPAIGN'].astype(str)
coupon_redempt['household_key'] = coupon_redempt['household_key'].astype(str)

transaction_data['household_key'] = transaction_data['household_key'].astype(str)
transaction_data['STORE_ID'] = transaction_data['STORE_ID'].astype(str)
transaction_data['PRODUCT_ID'] = transaction_data['PRODUCT_ID'].astype(str)
transaction_data['BASKET_ID'] = transaction_data['BASKET_ID'].astype(str)

coupon_redempt['PROMO_KEY'] = coupon_redempt['COUPON_UPC'] + '|' + coupon_redempt['CAMPAIGN']
coupon['PROMO_KEY'] = coupon['COUPON_UPC'] + '|' + coupon['CAMPAIGN']

product['PRODUCT_ID'] = product['PRODUCT_ID'].astype(str)
product['DEPARTMENT'] = product['DEPARTMENT'].astype(str)
product['MANUFACTURER'] = product['MANUFACTURER'].astype(str)


###merge

In [14]:
#creaaate new promo dataset | promo catlog
promo_catalog = coupon.merge(
    campaign_desc[['CAMPAIGN','START_DAY','END_DAY']],
    on='CAMPAIGN', how='left'
)[['PROMO_KEY','COUPON_UPC','CAMPAIGN','PRODUCT_ID','START_DAY','END_DAY']]

promo_catalog.head()


,PROMO_KEY,COUPON_UPC,CAMPAIGN,PRODUCT_ID,START_DAY,END_DAY
0,10000089061|4,10000089061,4,27160,372,404
1,10000089064|9,10000089064,9,27754,435,467
2,10000089073|12,10000089073,12,28897,477,509
3,51800009050|28,51800009050,28,28919,259,320
4,52100000076|25,52100000076,25,28929,659,691


In [15]:
hh_campaigns = campaign_table[['household_key','CAMPAIGN']].drop_duplicates()
hh_promos = hh_campaigns.merge(
    promo_catalog[['CAMPAIGN','PROMO_KEY','START_DAY','END_DAY']],
    on='CAMPAIGN', how='left'
).dropna(subset=['PROMO_KEY'])

hh_promos.head()

,household_key,CAMPAIGN,PROMO_KEY,START_DAY,END_DAY
0,27,26,51111037233|26,224,264
1,27,26,51111037233|26,224,264
2,27,26,51111037233|26,224,264
3,27,26,51111037233|26,224,264
4,27,26,51111037233|26,224,264


In [16]:
# dic represent which set of products are contained in each promotion
promo_to_products = (
    coupon[['COUPON_UPC','CAMPAIGN','PRODUCT_ID']]
    .assign(PROMO_KEY=lambda df: df['COUPON_UPC'] + '|' + df['CAMPAIGN'])
    .groupby('PROMO_KEY')['PRODUCT_ID']
    .apply(lambda s: set(s.astype(str)))
    .to_dict()
)

In [17]:
coupon_prod = coupon.merge(
    product[['PRODUCT_ID','DEPARTMENT','BRAND']],
    on='PRODUCT_ID', how='left'
).assign(PROMO_KEY=lambda df: df['COUPON_UPC'] + '|' + df['CAMPAIGN'])
coupon_prod['DEPARTMENT'] = coupon_prod['DEPARTMENT'].fillna('UNK')
coupon_prod['BRAND'] = coupon_prod['BRAND'].fillna('UNK')

promo_ohe = pd.get_dummies(
    coupon_prod[['PROMO_KEY','DEPARTMENT','BRAND']], columns=['DEPARTMENT','BRAND'], drop_first=False
).groupby('PROMO_KEY').max().astype(bool)

coupon_prod.head()


,COUPON_UPC,PRODUCT_ID,CAMPAIGN,PROMO_KEY,DEPARTMENT,BRAND
0,10000089061,27160,4,10000089061|4,UNK,UNK
1,10000089064,27754,9,10000089064|9,UNK,UNK
2,10000089073,28897,12,10000089073|12,UNK,UNK
3,51800009050,28919,28,51800009050|28,UNK,UNK
4,52100000076,28929,25,52100000076|25,UNK,UNK


In [18]:
promo_ohe

,DEPARTMENT_CHEF SHOPPE,DEPARTMENT_COSMETICS,DEPARTMENT_COUP/STR & MFG,DEPARTMENT_DAIRY DELI,DEPARTMENT_DELI,DEPARTMENT_DRUG GM,DEPARTMENT_FLORAL,DEPARTMENT_FROZEN GROCERY,DEPARTMENT_GARDEN CENTER,DEPARTMENT_GRO BAKERY,...,DEPARTMENT_RESTAURANT,DEPARTMENT_RX,DEPARTMENT_SALAD BAR,DEPARTMENT_SEAFOOD,DEPARTMENT_SEAFOOD-PCKGD,DEPARTMENT_TRAVEL & LEISUR,DEPARTMENT_UNK,BRAND_National,BRAND_Private,BRAND_UNK
PROMO_KEY,,,,,,,,,,,,,,,,,,,,,
10000085189|26,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
10000085190|26,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
10000085207|29,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
10000085319|30,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
10000085320|30,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58978500076|13,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
59108312076|23,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
59296110082|13,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True


In [19]:
promo_pop = coupon_redempt['PROMO_KEY'].value_counts().to_dict()

In [20]:
redempt_by_hh_day = (
    coupon_redempt.groupby(['household_key','DAY'])['PROMO_KEY']
    .apply(set).to_dict()
)

In [21]:
coupon_redempt= coupon_redempt.merge(
    coupon[['COUPON_UPC', 'PRODUCT_ID']],
    on='COUPON_UPC',
    how='left'
)

coupon= coupon.merge(
    campaign_desc[['START_DAY', 'END_DAY', 'CAMPAIGN']],
    on='CAMPAIGN',
    how='left'
)

coupon = coupon.merge(
    product[['PRODUCT_ID', 'DEPARTMENT', 'MANUFACTURER']],
    on='PRODUCT_ID',
    how='left'
)

### create timestamp

In [22]:
def create_timestamp(day, trans_time):
    base_date = datetime(2010, 3, 24)  # Arbitrary base date
    date_part = base_date + timedelta(days=int(day) - 1)

    # Convert TRANS_TIME to time
    hours = int(trans_time) // 100
    minutes = int(trans_time) % 100
    time_part = timedelta(hours=hours, minutes=minutes)

    return date_part + time_part

# Add timestamp to transaction_data
transaction_data['timestamp'] = transaction_data.apply(
    lambda row: create_timestamp(row['DAY'], row['TRANS_TIME']), axis=1
)

ts = transaction_data["timestamp"]
transaction_data["hour"]        = ts.dt.hour.astype("Int64")

transaction_data["dayofweek"]   = ts.dt.dayofweek.astype("Int64")    # 0=Mon .. 6=Sun
transaction_data["is_weekend"]  = transaction_data["dayofweek"].isin([5,6]).astype("int64")

transaction_data["day"]         = ts.dt.day.astype("Int64")
transaction_data["week"]        = ts.dt.isocalendar().week.astype("Int64")
transaction_data["month"]       = ts.dt.month.astype("Int64")
transaction_data["quarter"]     = ts.dt.quarter.astype("Int64")
transaction_data["year"]        = ts.dt.year.astype("Int64")

def make_part_of_day(hhmm_int):
    hh = hhmm_int // 100
    return (
        "night" if hh < 6 else
        "morning" if hh < 12 else
        "afternoon" if hh < 18 else
        "evening"
    )

transaction_data["part_of_day"] = transaction_data["TRANS_TIME"].astype(int).apply(make_part_of_day)
transaction_data = transaction_data.merge(
        product[['PRODUCT_ID', 'DEPARTMENT',"MANUFACTURER","BRAND"]],
        on='PRODUCT_ID',
        how='left'
    )

Merging transaction data

In [23]:
transaction_data = transaction_data.merge(
    coupon_redempt,
    on=['household_key', 'DAY', 'PRODUCT_ID'],
    how='left'
)

In [24]:
transaction_data

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,...,month,quarter,year,part_of_day,DEPARTMENT,MANUFACTURER,BRAND,COUPON_UPC,CAMPAIGN,PROMO_KEY
0,1130,26984905972,1,833715,2,0.34,31642,-0.32,1340,1,...,3,1,2010,afternoon,GROCERY,1002,National,NaN,NaN,NaN
1,1130,26984905972,1,866950,2,0.34,31642,-0.32,1340,1,...,3,1,2010,afternoon,GROCERY,1002,National,NaN,NaN,NaN
2,1130,26984905972,1,1022843,2,0.34,31642,-0.32,1340,1,...,3,1,2010,afternoon,GROCERY,1002,National,NaN,NaN,NaN
3,1130,26984905972,1,1048462,1,1.19,31642,-0.80,1340,1,...,3,1,2010,afternoon,GROCERY,1273,National,NaN,NaN,NaN
4,1130,26984905972,1,1071333,2,0.34,31642,-0.32,1340,1,...,3,1,2010,afternoon,GROCERY,1002,National,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328227,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,...,3,1,2012,afternoon,GROCERY,560,National,NaN,NaN,NaN
1328228,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,...,3,1,2012,afternoon,GROCERY,2398,National,NaN,NaN,NaN
1328229,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,...,3,1,2012,afternoon,GROCERY,2468,National,NaN,NaN,NaN
1328230,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,...,3,1,2012,afternoon,GROCERY,2390,National,NaN,NaN,NaN


In [50]:
RANDOM_STATE = 42
TOPK_CANDS = 20         # จำนวน candidates ต่อ query ที่จะป้อนเข้า ranker
EVAL_AT = [1,3,5,10]

In [51]:
transaction_data = transaction_data.sort_values(["household_key", "timestamp", "BASKET_ID"])
orders = (
    transaction_data.groupby(["household_key", "BASKET_ID"])
    .agg(items=("PRODUCT_ID", lambda s: set(s.tolist())),
         basket_day=("DAY", "min"))
    .reset_index()
)
orders

,household_key,BASKET_ID,items,basket_day
0,1,27601281299,"{912676, 7170493, 7431134, 5577022, 991024, 74...",51
1,1,27774192959,"{1055831, 997025, 1049998, 856942, 6773032, 95...",67
2,1,28024266849,"{841266, 1075074, 865178, 995904, 991024, 9527...",88
3,1,28106322445,"{5978656, 5577022, 7167962, 991580, 1078449, 8...",94
4,1,28235481967,"{5565840, 10149640, 887375, 856942, 5577022, 7...",101
...,...,...,...,...
124041,998,42031013564,"{849843, 981760, 899624, 845208, 995242}",691
124042,998,42049756099,{962568},693
124043,998,42063285897,"{843338, 865874, 12524086}",694
124044,998,42185598715,"{6533765, 1018085, 1049922}",703


## build candidate


In [52]:
def get_active_promos_for_household_on_day(hh: str, day: int) -> list[str]:
    rows = hh_promos[hh_promos['household_key'] == hh]
    rows = rows[(rows['START_DAY'] <= int(day)) & (int(day) <= rows['END_DAY'])]
    return rows['PROMO_KEY'].tolist()

# create candidate list for hh-id in specific day
def build_promo_candidates(household_key: str, user_hist_items: set[str], day: int, topk: int = TOPK_CANDS) -> list[str]:
    active = get_active_promos_for_household_on_day(household_key, day) #check if it active for hh_id in that day
    if not active:
        return []
    # score by overlap with user history + 0.001 x global popularity
    def score(pkey: str) -> float:
        prods = promo_to_products.get(pkey, set())
        overlap = len(prods & set(user_hist_items or []))
        pop = promo_pop.get(pkey, 0)
        return overlap + 0.001 * pop
    ranked = sorted(active, key=score, reverse=True)
    return ranked[:topk]

In [53]:
orders_day = (
    orders.sort_values(["household_key","basket_day","BASKET_ID"])
    .groupby(["household_key","basket_day"], as_index=False)
    .first()  # use only first basket_id on that day
    .rename(columns={"basket_day":"day_t"})
)

In [54]:
def _add_hist_exclusive_per_day(grp):
    hist = set()
    res = []
    for s in grp["items"]:
        res.append(hist.copy())
        hist |= s
    out = grp.copy()
    out["hist_exclusive"] = res
    return out

In [55]:
orders_fast = (
    orders_day.sort_values(["household_key","day_t"])
    .groupby("household_key", group_keys=False)
    .apply(_add_hist_exclusive_per_day)
    .reset_index(drop=True)
)

In [56]:
orders_fast

,household_key,day_t,BASKET_ID,items,hist_exclusive
0,1,51,27601281299,"{912676, 7170493, 7431134, 5577022, 991024, 74...",{}
1,1,67,27774192959,"{1055831, 997025, 1049998, 856942, 6773032, 95...","{1024128, 912676, 7170493, 7431134, 852014, 55..."
2,1,88,28024266849,"{841266, 1075074, 865178, 995904, 991024, 9527...","{912676, 7170493, 7431134, 1049998, 5577022, 9..."
3,1,94,28106322445,"{5978656, 5577022, 7167962, 991580, 1078449, 8...","{912676, 7170493, 7431134, 1049998, 5577022, 9..."
4,1,101,28235481967,"{5565840, 10149640, 887375, 856942, 5577022, 7...","{912676, 7170493, 7431134, 1049998, 5978656, 5..."
...,...,...,...,...,...
94228,998,683,41918011754,"{1091365, 1007136, 1134152, 15830211}","{912669, 1104205, 5995503, 821974, 12301100, 1..."
94229,998,691,42031013564,"{849843, 981760, 899624, 845208, 995242}","{912669, 1104205, 5995503, 821974, 12301100, 1..."
94230,998,693,42049756099,{962568},"{912669, 1104205, 5995503, 821974, 12301100, 1..."
94231,998,694,42063285897,"{843338, 865874, 12524086}","{912669, 1104205, 5995503, 821974, 12301100, 1..."


In [57]:
# check active promo
hh_to_promos = {
    hh: grp[["START_DAY","END_DAY","PROMO_KEY"]].to_numpy()
    for hh, grp in hh_promos.groupby("household_key", sort=False)
}

In [58]:
@lru_cache(maxsize=1_000_000)
def get_active_promos_tuple(hh: str, day: int) -> tuple:
    arr = hh_to_promos.get(hh)
    if arr is None:
        return ()
    d = int(day)
    mask = (arr[:,0] <= d) & (d <= arr[:,1])
    if not mask.any():
        return ()
    return tuple(arr[mask][:,2])

In [59]:
product_to_promos = defaultdict(set)
for pkey, prods in promo_to_products.items():
    for pid in prods:
        product_to_promos[pid].add(pkey)

# 3) แคช active ที่เรียงตามความนิยมไว้เติม fallback
@lru_cache(maxsize=1_000_000)
def get_active_promos_sorted_by_pop(hh: str, day: int) -> tuple:
    active = get_active_promos_tuple(hh, day)
    if not active:
        return ()
    # เรียงตาม popularity จากมากไปน้อย
    return tuple(sorted(active, key=lambda k: promo_pop.get(k, 0), reverse=True))

# Locals
empty_set = frozenset()
promo_pop_local = promo_pop
promo_to_products_local = promo_to_products
redempt_by_hh_day_local = redempt_by_hh_day
TOPK = TOPK_CANDS  # ปรับลงเป็น 20 ชั่วคราวได้ตอนเทสต์

# 4) Build columns
rng = np.random.RandomState(42)
col_hh, col_bid, col_promo, col_label = [], [], [], []

for r in orders_fast.itertuples(index=False):
    hh = r.household_key
    bid = r.BASKET_ID
    day_t = int(r.day_t)
    user_hist = r.hist_exclusive if isinstance(r.hist_exclusive, set) else set(r.hist_exclusive)

    active = get_active_promos_tuple(hh, day_t)
    if not active:
        continue
    active_set = set(active)

    # 4.1) ใช้ inverted index สร้างชุด candidate จากสินค้าที่ user เคยซื้อ (ลดขนาดอย่างมาก)
    cand_from_hist = set()
    # จำกัดขนาดประวัติ (เช่น 200 ชิ้นล่าสุด) เพื่อลดเวลา
    if len(user_hist) > 200:
        # แปลงเป็น list แล้วหยิบต้นๆ พอเป็นตัวแทน (hist_exclusive เป็น set จึงไม่มีลำดับที่แท้)
        user_hist_iter = list(user_hist)[:200]
    else:
        user_hist_iter = user_hist

    for pid in user_hist_iter:
        cand_from_hist |= product_to_promos.get(pid, empty_set)

    # เฉพาะโปรโมชันที่ active จริงในวันนั้น
    cand_from_hist &= active_set

    # 4.2) ให้คะแนนเฉพาะชุดที่ overlap (เล็กลงมาก) แล้วคัด TopK
    def score(pkey: str) -> float:
        prods = promo_to_products_local.get(pkey, empty_set)
        return (len(prods & user_hist) if prods else 0.0) + 0.001 * promo_pop_local.get(pkey, 0)

    if cand_from_hist:
        ranked_overlap = heapq.nlargest(TOPK, cand_from_hist, key=score)
    else:
        ranked_overlap = []

    # 4.3) เติม fallback ด้วย active ที่นิยมสูง (ไม่ต้องคำนวณ score เพิ่ม)
    if len(ranked_overlap) < TOPK:
        need = TOPK - len(ranked_overlap)
        fallback = []
        seen = set(ranked_overlap)
        for p in get_active_promos_sorted_by_pop(hh, day_t):
            if p not in seen:
                seen.add(p)
                fallback.append(p)
                if len(fallback) >= need:
                    break
        cands_base = ranked_overlap + fallback
    else:
        cands_base = ranked_overlap

    # 4.4) ใส่ positives (same-day) ให้อยู่ต้นๆ แล้ว dedupe
    positives = redempt_by_hh_day_local.get((hh, day_t), empty_set)
    if positives:
        pos_list = list(positives)
        if pos_list:
            seen = set()
            cands = []
            for p in pos_list:
                if p not in seen:
                    seen.add(p); cands.append(p)
            for p in cands_base:
                if p not in seen:
                    seen.add(p); cands.append(p)
            cands = cands[:TOPK]
        else:
            cands = cands_base[:TOPK]
    else:
        cands = cands_base[:TOPK]

    if not cands:
        continue
    rng.shuffle(cands)
    is_pos = positives.__contains__
    col_hh.extend([hh] * len(cands))
    col_bid.extend([int(bid)] * len(cands))
    col_promo.extend(cands)
    col_label.extend([1 if is_pos(p) else 0 for p in cands])


pair_df = pd.DataFrame({
    "household_key": col_hh,
    "BASKET_ID": col_bid,
    "PROMO_KEY": col_promo,
    "label": col_label
})
# 8 mins

In [60]:
# ตรวจ dtype key ให้ merge/group ได้เสมอ
def normalize_keys(df, keys):
    for k in keys:
        df[k] = df[k].astype(str)
    return df

# สร้างฐาน (pair_df + บริบทจาก orders_fast)
def build_base(pair_df, orders_fast):
    normalize_keys(pair_df, ['household_key','BASKET_ID'])
    normalize_keys(orders_fast, ['household_key','BASKET_ID'])

    base = pair_df.merge(
        orders_fast[['household_key','BASKET_ID','day_t','hist_exclusive']],
        on=['household_key','BASKET_ID'], how='left'
    )
    # guard
    assert {'household_key','BASKET_ID','PROMO_KEY','label','day_t','hist_exclusive'}.issubset(base.columns)
    base = base.reset_index(drop=True).copy()
    base['qid'] = base['household_key'].astype(str) + '_' + base['BASKET_ID'].astype(str)
    base['label'] = base['label'].astype(int)
    base['day_t'] = pd.to_numeric(base['day_t'], errors='coerce')
    return base


def make_scored_frame(model, Xv, base, valid_mask):
    vf = base.loc[valid_mask, ['qid','label']].copy()
    vf['score'] = model.predict(Xv)
    # break ties to remove order bias from candidate construction
    vf['score'] = vf['score'] + np.random.RandomState(42).normal(0, 1e-6, size=len(vf))
    vf['rank'] = vf.groupby('qid')['score'].rank(ascending=False, method='first')
    return vf

def hitrate_at_k(df, k=10):
    top = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    return (top.groupby('qid')['label'].max()).mean()

def recall_precision_at_k(df, k=10):
    got = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    pos_per_q = df.groupby('qid')['label'].sum()
    hit_per_q = got.groupby('qid')['label'].sum()
    recall = (hit_per_q / pos_per_q.replace(0, np.nan)).mean()
    precision = got.groupby('qid')['label'].mean().mean()
    return float(recall), float(precision)

def mrr_at_k(df, k=10):
    top = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    first_hit = top[top['label']==1].groupby('qid')['rank'].min()
    return (1.0/first_hit).fillna(0).mean()

def map_at_k(df, k=10):
    tops = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    ap = []
    for qid, sub in tops.groupby('qid'):
        if sub['label'].sum() == 0:
            continue
        sub = sub.sort_values('rank')
        cum_hits = sub['label'].cumsum()
        prec_i = cum_hits / np.arange(1, len(sub)+1)
        ap.append((prec_i * sub['label']).sum() / sub['label'].sum())
    return np.mean(ap) if ap else 0.0

In [61]:
# Build base table (no split)
feat_base = build_base(pair_df, orders_fast)
# If downstream code expects masks, make all-True placeholders
train_mask = np.ones(len(feat_base), dtype=bool)
valid_mask = np.ones(len(feat_base), dtype=bool)

In [62]:
from bisect import bisect_right

def build_recency_features_past_only(base, hh_pid_days):
    n = len(base)
    feat_min  = np.full(n, 9999, dtype=np.int32)
    feat_mean = np.full(n, 9999.0, dtype=np.float32)
    feat_seen = np.zeros(n, dtype=np.int32)
    for i, (hh, pkey, day_t) in enumerate(base[['household_key','PROMO_KEY','day_t']].itertuples(index=False)):
        prods = promo_to_products.get(pkey, None)
        if not prods:
            continue
        cutoff = int(day_t) - 1
        vals = []
        for pid in prods:
            arr = hh_pid_days.get((hh, pid))
            if arr is None or len(arr) == 0:
                continue
            j = bisect_right(arr, cutoff) - 1
            if j >= 0:
                dt = int(day_t) - int(arr[j])
                if dt > 0:
                    vals.append(dt)
        if vals:
            feat_min[i]  = min(vals)
            feat_mean[i] = float(np.mean(vals))
            feat_seen[i] = len(vals)
    return pd.DataFrame({'feat_min_recency': feat_min,
                         'feat_mean_recency': feat_mean,
                         'feat_seen_products': feat_seen}, index=base.index)

In [63]:
def build_recency_decay_overlap_past_only(base, hh_pid_days, halflifes=(3,7,14)):
    out = {f'overlap_decay_h{h}': np.zeros(len(base), dtype=np.float32) for h in halflifes}
    for i, (hh, pkey, day_t) in enumerate(base[['household_key','PROMO_KEY','day_t']].itertuples(index=False)):
        prods = promo_to_products.get(pkey, None)
        if not prods:
            continue
        cutoff = int(day_t) - 1
        if cutoff < 0:
            continue
        for h in halflifes:
            s = 0.0
            for pid in prods:
                arr = hh_pid_days.get((hh, pid))
                if arr is None or len(arr) == 0:
                    continue
                j = bisect_right(arr, cutoff) - 1
                if j >= 0:
                    dt = int(day_t) - int(arr[j])
                    if dt > 0:
                        s += 0.5 ** (dt / float(h))
            out[f'overlap_decay_h{h}'][i] = s
    return pd.DataFrame(out, index=base.index)

In [64]:
assert {'COUPON_UPC','CAMPAIGN','PRODUCT_ID'}.issubset(coupon.columns)
assert 'PRODUCT_ID' in product.columns

# PROMO_KEY
coupon = coupon.copy()
coupon['PROMO_KEY'] = coupon['COUPON_UPC'].astype(str) + '|' + coupon['CAMPAIGN'].astype(str)

# promo -> products
promo_to_products = (
    coupon[['PROMO_KEY','PRODUCT_ID']].drop_duplicates()
    .groupby('PROMO_KEY')['PRODUCT_ID'].apply(lambda s: set(s.dropna()))
    .to_dict()
)


promo_dept_set = (
    coupon_prod[['PROMO_KEY','DEPARTMENT']].drop_duplicates()
    .groupby('PROMO_KEY')['DEPARTMENT'].apply(lambda s: set(s.dropna()))
    .to_dict()
)

promo_brand_set = (
    coupon_prod[['PROMO_KEY','BRAND']].drop_duplicates()
    .groupby('PROMO_KEY')['BRAND'].apply(lambda s: set(s.dropna()))
    .to_dict()
)

def build_user_affinity_features_past_only(base, hh_dept_days, hh_brand_days):
    u_dept = np.zeros(len(base), dtype=np.int32)
    u_brand = np.zeros(len(base), dtype=np.int32)
    for i, (hh, pkey, day_t) in enumerate(base[['household_key','PROMO_KEY','day_t']].itertuples(index=False)):
        cutoff = int(day_t) - 1
        for d in promo_dept_set.get(pkey, set()):
            arr = hh_dept_days.get((hh, d))
            if arr is not None and len(arr):
                u_dept[i] += bisect_right(arr, cutoff)
        for b in promo_brand_set.get(pkey, set()):
            arr = hh_brand_days.get((hh, b))
            if arr is not None and len(arr):
                u_brand[i] += bisect_right(arr, cutoff)
    return pd.DataFrame({'feat_user_dept_aff': u_dept, 'feat_user_brand_aff': u_brand}, index=base.index)


In [65]:
def build_immediate_signals_past_only(base, hh_pid_last, windows=(1,3,7)):
    out = {f'seen_in_{w}d': np.zeros(len(base), dtype=np.uint8) for w in windows}
    out['short_overlap_7d'] = np.zeros(len(base), dtype=np.int16)
    for i, (hh, pkey, day_t) in enumerate(base[['household_key','PROMO_KEY','day_t']].itertuples(index=False)):
        prods = promo_to_products.get(pkey, set())
        cutoff = int(day_t) - 1
        flags = {w: 0 for w in windows}
        cnt7 = 0
        for pid in prods:
            dlast = hh_pid_last.get((hh, pid))
            if dlast is None or dlast > cutoff:
                continue
            dt = cutoff - int(dlast)
            for w in windows:
                if dt <= w:
                    flags[w] = 1
            if dt <= 7:
                cnt7 += 1
        for w in windows:
            out[f'seen_in_{w}d'][i] = flags[w]
        out['short_overlap_7d'][i] = cnt7
    return pd.DataFrame(out, index=base.index)

In [66]:
# เตรียม daily (เหมือนเดิม)
pair_with_day = pair_df[['PROMO_KEY','label']].copy()
pair_with_day['day_t'] = feat_base['day_t'].astype(int).values

daily = (pair_with_day
         .groupby(['PROMO_KEY','day_t'])['label']
         .agg(offer_ct='count', pos_ct='sum')
         .reset_index())
daily['PROMO_KEY'] = daily['PROMO_KEY'].astype(str)
daily['day_t']     = daily['day_t'].astype(int)
daily = daily.sort_values(['PROMO_KEY','day_t'], kind='mergesort', ascending=[True, True]).reset_index(drop=True)
daily['cum_offer'] = daily.groupby('PROMO_KEY')['offer_ct'].cumsum()
daily['cum_pos']   = daily.groupby('PROMO_KEY')['pos_ct'].cumsum()

right = (daily.rename(columns={'day_t':'key_day'})[['PROMO_KEY','key_day','cum_offer','cum_pos']])

# ฝั่งซ้าย + เก็บ index เดิม
fb = feat_base[['PROMO_KEY','day_t']].copy()
fb['orig_idx'] = fb.index
fb['PROMO_KEY'] = fb['PROMO_KEY'].astype(str)
fb['day_t_minus1'] = (pd.to_numeric(fb['day_t'], errors='coerce') - 1).astype('Int64')
left = fb.dropna(subset=['day_t_minus1']).copy()
left['day_t_minus1'] = left['day_t_minus1'].astype(int)

# ทำ asof ต่อกลุ่ม PROMO_KEY (หลีกเลี่ยงเงื่อนไข sort ข้ามกลุ่ม)
merged_parts = []
for k, gleft in left.groupby('PROMO_KEY', sort=False):
    gright = right[right['PROMO_KEY'] == k]
    if gright.empty:
        g = gleft.copy()
        g['cum_offer'] = np.nan
        g['cum_pos']   = np.nan
    else:
        g = pd.merge_asof(
            gleft.sort_values('day_t_minus1', kind='mergesort'),
            gright.sort_values('key_day',      kind='mergesort'),
            left_on='day_t_minus1', right_on='key_day',
            direction='backward', allow_exact_matches=True
        )
    merged_parts.append(g)

merged = pd.concat(merged_parts, ignore_index=True)

# ประกอบกลับตาม feat_base.index
feat_popularity = pd.DataFrame(index=feat_base.index, data={'promo_offer_ct':0, 'promo_pos_ct':0}, dtype='int32')
feat_popularity.loc[merged['orig_idx'].to_numpy(), 'promo_offer_ct'] = merged['cum_offer'].fillna(0).astype('int32').to_numpy()
feat_popularity.loc[merged['orig_idx'].to_numpy(), 'promo_pos_ct']   = merged['cum_pos'].fillna(0).astype('int32').to_numpy()
feat_popularity['feat_promo_pos_rate']       = (feat_popularity['promo_pos_ct'] / feat_popularity['promo_offer_ct'].replace(0, np.nan)).fillna(0).astype('float32')
feat_popularity['feat_promo_offer_ct_log1p'] = np.log1p(feat_popularity['promo_offer_ct'])
feat_popularity['feat_promo_pos_ct_log1p']   = np.log1p(feat_popularity['promo_pos_ct'])

In [67]:
feat_popularity

,promo_offer_ct,promo_pos_ct,feat_promo_pos_rate,feat_promo_offer_ct_log1p,feat_promo_pos_ct_log1p
0,12,0,0.000000,2.564949,0.000000
1,11,0,0.000000,2.484907,0.000000
2,11,0,0.000000,2.484907,0.000000
3,13,0,0.000000,2.639057,0.000000
4,13,0,0.000000,2.639057,0.000000
...,...,...,...,...,...
689054,1449,3,0.002070,7.279319,1.386294
689055,758,1,0.001319,6.632002,0.693147
689056,1474,6,0.004071,7.296413,1.945910
689057,1114,2,0.001795,7.016610,1.098612


In [68]:
# ต้องได้ True ทั้งหมด
print((left[['PROMO_KEY','day_t_minus1']]
       .sort_values(['PROMO_KEY','day_t_minus1'], kind='mergesort')
       .equals(left[['PROMO_KEY','day_t_minus1']])))
print(left['day_t_minus1'].isna().any(), right['key_day'].isna().any())
print(left['PROMO_KEY'].dtype, right['PROMO_KEY'].dtype)

False
False False
object object


In [69]:
def build_coverage_features(base):
    out = base[['PROMO_KEY']].copy()
    out['feat_num_promo_products'] = out['PROMO_KEY'].map(lambda k: len(promo_to_products.get(k, set()))).astype(int)
    out['feat_num_promo_departments'] = out['PROMO_KEY'].map(lambda k: len(promo_dept_set.get(k, set()))).astype(int)
    out['feat_num_promo_brands'] = out['PROMO_KEY'].map(lambda k: len(promo_brand_set.get(k, set()))).astype(int)
    return out[['feat_num_promo_products','feat_num_promo_departments','feat_num_promo_brands']].set_index(base.index)

feat_coverage = build_coverage_features(feat_base)

In [70]:
def build_promo_ohe_features(base):
    ohe = promo_ohe.copy()
    if ohe.index.name != 'PROMO_KEY':
        if 'PROMO_KEY' in ohe.columns:
            ohe = ohe.set_index('PROMO_KEY')
    out = ohe.reindex(base['PROMO_KEY']).fillna(0)
    for c in out.columns:
        if out[c].dtype == bool:
            out[c] = out[c].astype(np.uint8)
        elif out[c].dtype == object:
            out[c] = pd.to_numeric(out[c], errors='coerce').fillna(0).astype(np.uint8)
    out.index = base.index
    return out

In [71]:
def build_temporal_features(base):
    out = base[['day_t']].copy()
    day_t = pd.to_numeric(out['day_t'], errors='coerce').fillna(0).astype(int)
    out['feat_dayofweek'] = (day_t % 7).astype(np.int8)
    out['feat_is_weekend'] = out['feat_dayofweek'].isin([5,6]).astype(np.int8)
    return out[['feat_dayofweek','feat_is_weekend']].set_index(base.index)

feat_temporal = build_temporal_features(feat_base)

In [72]:
tx = transaction_data[['household_key','PRODUCT_ID','DEPARTMENT','BRAND','DAY']].copy()
tx['DEPARTMENT'] = tx['DEPARTMENT'].fillna('UNK').astype(str)
tx['BRAND']      = tx['BRAND'].fillna('UNK').astype(str)

# (hh, pid) -> np.array(days)
tx_pid = tx.sort_values(['household_key','PRODUCT_ID','DAY'])
hh_pid_days = tx_pid.groupby(['household_key','PRODUCT_ID'], sort=False)['DAY'] \
                    .apply(lambda s: s.to_numpy(dtype=np.int32)).to_dict()

# last day เป็น dict ธรรมดา
hh_pid_last = {k: int(v[-1]) for k, v in hh_pid_days.items() if len(v) > 0}

# (hh, dept) / (hh, brand) -> np.array(days)
tx_dept = tx.sort_values(['household_key','DEPARTMENT','DAY'])
tx_brand= tx.sort_values(['household_key','BRAND','DAY'])
hh_dept_days = tx_dept.groupby(['household_key','DEPARTMENT'], sort=False)['DAY'] \
                      .apply(lambda s: s.to_numpy(dtype=np.int32)).to_dict()
hh_brand_days= tx_brand.groupby(['household_key','BRAND'], sort=False)['DAY'] \
                      .apply(lambda s: s.to_numpy(dtype=np.int32)).to_dict()

In [84]:
# --- builder แบบ past-only ---
def build_similarity_features_past_only(part, promo_to_products, hist_col='hist_until_t'):
    # ใช้ hist_until_t ถ้ามี ไม่งั้น fallback hist_exclusive
    if hist_col not in part.columns:
        hist_col = 'hist_exclusive'
    hist_series = part[hist_col].apply(lambda s: s if isinstance(s, set) else (set(s) if s is not None else set()))
    promo_sets = part['PROMO_KEY'].map(lambda k: promo_to_products.get(k, set()))

    hist_sizes = hist_series.apply(len)
    overlaps   = promo_sets.combine(hist_series, lambda a,b: len(a & b)).astype('int16')
    union_sz   = promo_sets.combine(hist_series, lambda a,b: len(a | b)).astype('int32')

    out = pd.DataFrame({
        'feat_hist_size': hist_sizes.astype('int32'),
        'feat_overlap_cnt': overlaps,
        'feat_overlap_ratio': (overlaps / hist_sizes.replace(0, np.nan)).fillna(0.0).astype('float32'),
        'feat_jaccard': (overlaps / union_sz.replace(0, np.nan)).fillna(0.0).astype('float32'),
    }, index=part.index)
    return out



In [85]:
def build_in_chunks(base, builder, chunk_size=200_000, **kwargs):
    out = []
    for start in range(0, len(base), chunk_size):
        part = base.iloc[start:start+chunk_size]
        out.append(builder(part, **kwargs))
    return pd.concat(out, axis=0).reindex(base.index)

# ใช้งาน: (เลือก halflifes น้อยลงเพื่อลดเวลา แล้วค่อยเพิ่มทีหลัง)
feat_recency        = build_in_chunks(feat_base, build_recency_features_past_only, hh_pid_days=hh_pid_days)
feat_recency_decay  = build_in_chunks(feat_base, build_recency_decay_overlap_past_only,   hh_pid_days=hh_pid_days, halflifes=(3,5,7))
feat_affinity       = build_in_chunks(feat_base, build_user_affinity_features_past_only,        hh_dept_days=hh_dept_days, hh_brand_days=hh_brand_days)
feat_immediate      = build_in_chunks(feat_base, build_immediate_signals_past_only,       hh_pid_last=hh_pid_last, windows=(3,7))
# สร้าง feat_similarity ด้วย build_in_chunks เดิม
feat_similarity = build_in_chunks(
    feat_base,
    build_similarity_features_past_only,
    promo_to_products=promo_to_products,
    hist_col='hist_until_t'  # ถ้ายังไม่มี ให้เปลี่ยนเป็น 'hist_exclusive'
)

feat_coverage       = build_coverage_features(feat_base)
feat_temporal       = build_temporal_features(feat_base)
feat_promo_ohe      = build_promo_ohe_features(feat_base)

# 17 mins

In [75]:
feat_base.head()

,household_key,BASKET_ID,PROMO_KEY,label,day_t,hist_exclusive,qid
0,1,31172831466,54850010009|29,0,282,"{1084555, 866227, 1042564, 5564226, 9655175, 7...",1_31172831466
1,1,31172831466,57800000033|29,0,282,"{1084555, 866227, 1042564, 5564226, 9655175, 7...",1_31172831466
2,1,31172831466,54300021057|29,0,282,"{1084555, 866227, 1042564, 5564226, 9655175, 7...",1_31172831466
3,1,31172831466,51410010050|29,0,282,"{1084555, 866227, 1042564, 5564226, 9655175, 7...",1_31172831466
4,1,31172831466,54300016033|29,0,282,"{1084555, 866227, 1042564, 5564226, 9655175, 7...",1_31172831466


In [86]:
F_parts = [
    feat_recency, feat_recency_decay, feat_affinity, feat_immediate,
    feat_popularity,
    feat_coverage, feat_temporal, feat_promo_ohe,feat_similarity,
]
F_all = pd.concat([f for f in F_parts if f is not None], axis=1).reindex(feat_base.index)

In [78]:
F_all.head()

,feat_min_recency,feat_mean_recency,feat_seen_products,overlap_decay_h3,overlap_decay_h5,overlap_decay_h7,feat_user_dept_aff,feat_user_brand_aff,seen_in_3d,seen_in_7d,...,DEPARTMENT_RESTAURANT,DEPARTMENT_RX,DEPARTMENT_SALAD BAR,DEPARTMENT_SEAFOOD,DEPARTMENT_SEAFOOD-PCKGD,DEPARTMENT_TRAVEL & LEISUR,DEPARTMENT_UNK,BRAND_National,BRAND_Private,BRAND_UNK
0,8,34.0,4,0.182293,0.473468,0.757750,390,483,0,0,...,0,0,0,0,0,0,1,1,0,1
1,9999,9999.0,0,0.000000,0.000000,0.000000,390,483,0,0,...,0,0,0,0,0,0,1,1,0,1
2,9999,9999.0,0,0.000000,0.000000,0.000000,390,483,0,0,...,0,0,0,0,0,0,1,1,0,1
3,8,8.0,1,0.157490,0.329877,0.452862,390,483,0,0,...,0,0,0,0,0,0,1,1,0,1
4,9999,9999.0,0,0.000000,0.000000,0.000000,390,483,0,0,...,0,0,0,0,0,0,1,1,0,1


In [87]:
cols_keys = ['household_key','BASKET_ID','PROMO_KEY','day_t','qid','label']
df_keys = feat_base[cols_keys].copy()
df_keys['label'] = df_keys['label'].astype(np.int8)
df_keys['day_t'] = pd.to_numeric(df_keys['day_t'], errors='coerce').fillna(0).astype(np.int32)
df_out = pd.concat([df_keys, F_all], axis=1)

df_out.head()

,household_key,BASKET_ID,PROMO_KEY,day_t,qid,label,feat_min_recency,feat_mean_recency,feat_seen_products,overlap_decay_h3,...,DEPARTMENT_SEAFOOD-PCKGD,DEPARTMENT_TRAVEL & LEISUR,DEPARTMENT_UNK,BRAND_National,BRAND_Private,BRAND_UNK,feat_hist_size,feat_overlap_cnt,feat_overlap_ratio,feat_jaccard
0,1,31172831466,54850010009|29,282,1_31172831466,0,8,34.0,4,0.182293,...,0,0,1,1,0,1,288,4,0.013889,0.011173
1,1,31172831466,57800000033|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0,0,1,1,0,1,288,0,0.000000,0.000000
2,1,31172831466,54300021057|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0,0,1,1,0,1,288,0,0.000000,0.000000
3,1,31172831466,51410010050|29,282,1_31172831466,0,8,8.0,1,0.157490,...,0,0,1,1,0,1,288,1,0.003472,0.003367
4,1,31172831466,54300016033|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0,0,1,1,0,1,288,0,0.000000,0.000000


In [88]:
from pathlib import Path
out_dir = Path('Dataset'); out_dir.mkdir(parents=True, exist_ok=True)

df_out.to_parquet(out_dir / 'features_all.parquet', index=False)  # ต้องมี pyarrow หรือ fastparquet
df_out.to_csv(out_dir / 'features_all.csv', index=False)

print("Saved:", out_dir / 'features_all.parquet', "|", out_dir / 'features_all.csv')
print("Shape:", df_out.shape)

Saved: Dataset/features_all.parquet | Dataset/features_all.csv
Shape: (689059, 62)


In [90]:
df_out.columns

Index(['household_key', 'BASKET_ID', 'PROMO_KEY', 'day_t', 'qid', 'label',
       'feat_min_recency', 'feat_mean_recency', 'feat_seen_products',
       'overlap_decay_h3', 'overlap_decay_h5', 'overlap_decay_h7',
       'feat_user_dept_aff', 'feat_user_brand_aff', 'seen_in_3d', 'seen_in_7d',
       'short_overlap_7d', 'promo_offer_ct', 'promo_pos_ct',
       'feat_promo_pos_rate', 'feat_promo_offer_ct_log1p',
       'feat_promo_pos_ct_log1p', 'feat_num_promo_products',
       'feat_num_promo_departments', 'feat_num_promo_brands', 'feat_dayofweek',
       'feat_is_weekend', 'DEPARTMENT_CHEF SHOPPE', 'DEPARTMENT_COSMETICS',
       'DEPARTMENT_COUP/STR & MFG', 'DEPARTMENT_DAIRY DELI', 'DEPARTMENT_DELI',
       'DEPARTMENT_DRUG GM', 'DEPARTMENT_FLORAL', 'DEPARTMENT_FROZEN GROCERY',
       'DEPARTMENT_GARDEN CENTER', 'DEPARTMENT_GRO BAKERY',
       'DEPARTMENT_GROCERY', 'DEPARTMENT_HBC', 'DEPARTMENT_MEAT',
       'DEPARTMENT_MEAT-PCKGD', 'DEPARTMENT_MISC SALES TRAN',
       'DEPARTMENT_MISC.